In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
df_train=pd.read_csv("/kaggle/input/titanic/train.csv")
df_train

In [ ]:
df_test=pd.read_csv("/kaggle/input/titanic/test.csv")
df_test

In [ ]:
PsgId=df_test['PassengerId']
PsgId.shape

In [ ]:
df_train.describe().T

In [ ]:
df_train.select_dtypes(include='object').describe().T

In [ ]:
df_train.isnull().sum()

In training dataset, we have following missing values-

Age=177 missing values

Cabin=687 missing values

Embarked=2 missing values


In [ ]:
df_test.describe().T

In [ ]:
df_test.select_dtypes(include='object').describe().T

In [ ]:
df_test.isnull().sum()

In test data set, we have following missig values

Age=86 missing values

Cabin=327 missing values

Fare=1 missing values


**Handling missing values**

In [ ]:
df_train.Age.describe()

In [ ]:
df_train.Age=df_train.Age.fillna(df_train.Age.median())

In [ ]:
df_train.Age.describe()

In [ ]:
df_test.Age=df_test.Age.fillna(df_test.Age.median())

In [ ]:
df_test.Age.describe()

In [ ]:
df_train.drop('Cabin', axis=1, inplace=True)

In [ ]:
df_test.drop('Cabin', axis=1, inplace=True)

In [ ]:
df_test.Fare.describe()

In [ ]:
df_test.Fare=df_test.Fare.fillna(df_test.Fare.median())

In [ ]:
df_test.Fare.describe()

In [ ]:
df_train.Embarked=df_train.Embarked.fillna(df_train.Embarked.mode()[0])

**Feature Engineering**

In [ ]:
df_train['FamilySize'] = df_train['SibSp'] + df_train['Parch']+1

In [ ]:
df_test['FamilySize'] = df_test['SibSp'] + df_test['Parch']+1

In [ ]:
df_train['GenderClass'] = df_train.apply(lambda x: 'child' if x['Age'] < 15 else x['Sex'],axis=1)

In [ ]:
df_test['GenderClass'] = df_test.apply(lambda x: 'child' if x['Age'] < 15 else x['Sex'],axis=1)

In [ ]:
df_train.head()

In [ ]:
df_train.drop(['Name','Sex','Ticket','PassengerId','SibSp','Parch'], axis=1, inplace=True)

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_test.drop(['PassengerId','Name','Sex','SibSp','Parch','Ticket'], axis=1, inplace=True)

In [ ]:
df_test.head()

In [ ]:
df_train.dtypes

In [ ]:
#df_train=pd.get_dummies(df_train,columns=['Embarked','GenderClass'], axis=1)

In [ ]:
df_train.Embarked.head()

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

df_train['Embarked'] = le.fit_transform(df_train['Embarked'])

In [ ]:
df_train.Embarked.head()

In [ ]:
df_train['GenderClass'] = le.fit_transform(df_train['GenderClass'])

In [ ]:
df_train.head()

In [ ]:
df_train.shape

In [ ]:
df_train.dtypes

In [ ]:
df_test.dtypes

In [ ]:
df_test['Embarked'] = le.fit_transform(df_test['Embarked'])
df_test['GenderClass'] = le.fit_transform(df_test['GenderClass'])

In [ ]:
df_test.dtypes

**Visualization**

In [ ]:
df_train.head()

In [ ]:
df_test.head()

Checking for distribution

In [ ]:
sns.pairplot(df_train)

In [ ]:
sns.pairplot(df_test)

From the above pairplots for train and test data, we can observe that the distribution is not uniform and rather skewed
So, we need to uniform the data later before doing any modelling

In [ ]:
corr=df_train.corr()
sns.heatmap(corr)

In [ ]:
plt.figure(figsize=(8, 12))
df_train.corr()['Survived'].sort_values().plot(kind='barh')

In [ ]:
corr=df_test.corr()
sns.heatmap(corr)

In [ ]:
num_columns=df_train.select_dtypes(include=['float','int']).columns.to_list()
num_columns

In [ ]:
df_train.describe().T

In [ ]:
fig, ax=plt.subplots(len(num_columns),1,figsize=(8,30))
for i,axes in enumerate(ax):
  sns.kdeplot(df_train[num_columns[i]], ax=axes)
  axes.set_title(num_columns[i])

fig.tight_layout()

In [ ]:
num_columns=df_test.select_dtypes(include=['float','int']).columns.to_list()
num_columns

In [ ]:
fig, ax=plt.subplots(len(num_columns),1,figsize=(8,30))
for i,axes in enumerate(ax):
  sns.kdeplot(df_test[num_columns[i]], ax=axes)
  axes.set_title(num_columns[i])

fig.tight_layout()

In [ ]:
y=df_train.Survived

In [ ]:
y.shape

In [ ]:
X=df_train.drop('Survived',axis=1)

In [ ]:
X.shape

In [ ]:
X_test=df_test
X_test.shape

In [ ]:
plt.figure(figsize=(8,8))
sns.distplot(y)
plt.title("Survived distribution before normalization")

It is obvioius of no uniform distribution as Not Survived are more as compared to Survived

In [ ]:
from scipy.stats import kurtosis, skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

In [ ]:
"""
num_feat=X.dtypes[X.dtypes!='object'].index
skewed_feats = X[num_feat].apply(lambda x: skew(x)).sort_values(ascending=False)
high_skew = skewed_feats[abs(skewed_feats) > 0.5]
skew_index = high_skew.index
for i in skew_index:
    X[i] = boxcox1p(X[i], boxcox_normmax(X[i] + 1))
"""

In [ ]:
X.shape

In [ ]:
X.head()

In [ ]:
num_columns=X.select_dtypes(include=['float','int']).columns.to_list()
fig, ax=plt.subplots(len(num_columns),1,figsize=(8,30))
for i,axes in enumerate(ax):
  sns.kdeplot(X[num_columns[i]], ax=axes)
  axes.set_title(num_columns[i])

fig.tight_layout()

In [ ]:
"""
num_feat=X_test.dtypes[X_test.dtypes!='object'].index
skewed_feats = X_test[num_feat].apply(lambda x: skew(x)).sort_values(ascending=False)
high_skew = skewed_feats[abs(skewed_feats) > 0.5]
skew_index = high_skew.index
for i in skew_index:
    X_test[i] = boxcox1p(X_test[i], boxcox_normmax(X_test[i] + 1))
"""

In [ ]:
X_test.head()

In [ ]:
num_columns=X_test.select_dtypes(include=['float','int']).columns.to_list()
fig, ax=plt.subplots(len(num_columns),1,figsize=(8,30))
for i,axes in enumerate(ax):
  sns.kdeplot(X_test[num_columns[i]], ax=axes)
  axes.set_title(num_columns[i])

fig.tight_layout()

**Modelling the data using Logistic Regression base model****

In [ ]:
y.unique()

In [ ]:
y.shape

In [ ]:
X.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X,y) 

In [ ]:
y_pred_train = logreg.predict(X)

In [ ]:
y_pred_train.shape

In [ ]:
y.shape

In [ ]:
from sklearn.metrics import accuracy_score
print('Accuracy score for train data is:', accuracy_score(y,y_pred_train))

In [ ]:
preds1 = np.where(logreg.predict_proba(X)[:,1]> 0.75,1,0)
print('Accuracy score for train data is:', accuracy_score(y,y_pred_train))

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix = pd.DataFrame(confusion_matrix(y,y_pred_train))

confusion_matrix.index = ['Actual Died','Actual Survived']
confusion_matrix.columns = ['Predicted Died','Predicted Survived']
print(confusion_matrix)

In [ ]:
from sklearn.metrics import precision_score
precision1 = precision_score(y,y_pred_train)
print('Precision score for train data is:', precision1)

In [ ]:
from sklearn.metrics import recall_score
print('Recall score for train data is:',recall_score(y,y_pred_train)) 

**Using all models**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
MLA=[
    RandomForestClassifier(),
    DecisionTreeClassifier(),
    LogisticRegression()
]


In [ ]:
MLA_columns = ['MLA Name', 'MLA Parameters','MLA Train Accuracy Mean', 'MLA Test Accuracy Mean', 'MLA Test Accuracy 3*STD' ,'MLA Time', 'TrainTestDifference']
MLA_compare = pd.DataFrame(columns = MLA_columns)

MLA_predict=y.copy()

row_index = 0
Feature_Importance = {}

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:

for alg in MLA:
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())

    print(X.shape)
    print(y.shape)
    cv_results = cross_validate(alg, X, y, cv=3, return_train_score=True,scoring='accuracy')
    print(X.shape)
    print(y.shape)
    
    MLA_compare.loc[row_index, 'MLA Time'] = cv_results['fit_time'].mean()
    MLA_compare.loc[row_index, 'MLA Train Accuracy Mean'] = cv_results['train_score'].mean()
    MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = cv_results['test_score'].mean() 
    MLA_compare.loc[row_index, 'MLA Test Accuracy 3*STD'] = cv_results['test_score'].std()*3

    alg.fit(X,y)
    print(alg)

    try:
        Feature_Importance[MLA_name] = alg.feature_importances_
    except AttributeError:
        pass

    MLA_predict[MLA_name] = alg.predict(X)

    row_index+=1


In [ ]:
MLA_compare.sort_values(by = ['MLA Test Accuracy Mean'], ascending = False, inplace = True)
MLA_compare['Difference'] = (MLA_compare['MLA Test Accuracy Mean']-MLA_compare['MLA Train Accuracy Mean'])*100
MLA_compare

In [ ]:
Feature_Importance

In [ ]:
MLA_compare.sort_values(by="Difference",ascending=False)

In [ ]:
logreg=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
logreg.fit(X,y)

In [ ]:
y_pred_test=logreg.predict(X_test)

In [ ]:
y_pred_test.shape

In [ ]:
y_pred_test

In [ ]:
PsgId

In [ ]:
pd.DataFrame(zip(PsgId,y_pred_test),)

In [ ]:
sub = pd.DataFrame()
sub['PassengerId'] = PsgId
sub['Survived'] = y_pred_test
sub.to_csv('submission.csv',index=False)